In [1]:
import pandas as pd
from pathlib import Path


In [4]:
# todo - data loader script
#     [x] fix index (ticker instead of unnamed 0)
#     [] get currency


# todo - cleaning
#     [] calculate metrics to script

# todo - analysis
#     [] 5/10 yr sales, fcf, div, sharecount growth
#     [] 5/10 year roic

# todo - dashboard
#     [] top 10 for growth rates
#     [] top 10 for roic
#     [] top 10 for hindsight dcf valuation



In [79]:
# data processing functions
def load_data():
    # load data
    data_path = Path(Path().absolute().parent, 'data', 'financials.csv')
    df = pd.read_csv(data_path)

    # drop empty columns
    df = df[(df.period_end_date != '0')]

    # rename columns
    df.rename(columns={'Unnamed: 0':'ticker'}, inplace=True)

    # extract month/yeat
    df['period_end_year'] = df.period_end_date.str.split('-',expand=True)[0].astype('int64')
    df['period_end_month'] = df.period_end_date.str.split('-',expand=True)[1].astype('int64')

    # non int fileds to index 
    df = df.set_index(['ticker','period_end_date', 'period_end_month', 'period_end_year', 'period_end_price'])

    # convert to mil
    df = df / 1000000

    # return columns
    df = df.reset_index()

    # calculate metrics
    df['fcf'] = df.cf_cfo - df.cfi_ppe_net - df.cfi_intangibles_net

    return df


# calculate metrics functions
def calc_gross_margin(df):
    return df.groupby(['ticker', 'period_end_year']).apply(lambda x: x.gross_profit / x.revenue )


def calc_operating_margin(df):
    return df.groupby(['ticker', 'period_end_year']).apply(lambda x: x.operating_income / x.revenue )
    

def calc_net_margin(df):
    return df.groupby(['ticker', 'period_end_year']).apply(lambda x: x.net_income / x.revenue )


def calc_fcf_margin(df):
    return df.groupby(['ticker', 'period_end_year']).apply(lambda x: x.fcf / x.revenue)


def calc_pe_ratio(df):
    return df.groupby(['ticker', 'period_end_year']).apply(lambda x: x.market_cap / x.net_income)


def calc_fcf_yield(df):
    return df.groupby(['ticker', 'period_end_year']).apply(lambda x: x.fcf / x.market_cap)

# fucntion broken, needs to be fixed
def calc_roic(df):
    return df.groupby(['ticker', 'period_end_year']).apply(lambda x: x.fcf  / (x.lt_debt + x.noncurrent_capital_leases + x.total_equity - x.cash_and_equiv) )


def calc_roe(df):
    return df.groupby(['ticker', 'period_end_year']).apply(lambda x: x.fcf / x.total_assets )


def calc_roa(df):
    return df.groupby(['ticker', 'period_end_year']).apply(lambda x: x.fcf /  x.total_equity )

def calc_roce(df):
    return df.groupby(['ticker', 'period_end_year']).apply(lambda x: (x.net_income + x.income_tax + x.interest_expense) /  (x.total_assets - x.total_current_liabilities) )



def calculate_metrics(df, metric_names, functions):
    
    temp_dict = dict(zip(metric_names, functions))
    
    metric_values = []
    
    for m, f in temp_dict.items():
        metric_values.append(f(df))
    
    data = dict(zip(metric_names, metric_values))
    m_df = pd.DataFrame(data).dropna()
    m_df = m_df.reset_index().drop(columns=('level_2'))
    
    return m_df 


def get_calc_function_names(functions):
    clean_names = []

    for f in functions:

        word_list = f.__name__.split('_')

        name_words = []
        for w in word_list:
            if w != 'calc':
                name_words.append(w)
        clean_names.append('_'.join(name_words))
    return clean_names


In [80]:
# run

df = load_data()


calc_functions = [calc_gross_margin, calc_operating_margin, calc_net_margin, calc_fcf_margin, calc_pe_ratio, calc_fcf_yield, calc_roic, calc_roe, calc_roa, calc_roce]
names = get_calc_function_names(calc_functions) 

m_df = calculate_metrics(df, names, calc_functions)


In [81]:
df.columns

Index(['ticker', 'period_end_date', 'period_end_month', 'period_end_year',
       'period_end_price', 'revenue', 'cogs', 'gross_profit', 'sga', 'rnd',
       'operating_income', 'income_tax', 'interest_expense', 'net_income',
       'shares_diluted', 'cash_and_equiv', 'receivables', 'inventories',
       'total_current_assets', 'equity_and_other_investments', 'ppe_net',
       'intangible_assets', 'goodwill', 'total_assets', 'accounts_payable',
       'tax_payable', 'st_debt', 'total_current_liabilities', 'lt_debt',
       'noncurrent_capital_leases', 'total_liabilities', 'retained_earnings',
       'total_equity', 'cf_cfo', 'cfi_ppe_net', 'cfi_acquisitions_net',
       'cfi_investment_net', 'cfi_intangibles_net', 'cff_common_stock_net',
       'cff_debt_net', 'cff_dividend_paid', 'cf_cff', 'market_cap', 'fcf'],
      dtype='object')

In [89]:
roic = m_df.groupby('ticker')['roic'].transform(lambda x: x.rolling(5, 1).mean())
dict(zip(roic, m_df.ticker.values))

{1.146141744121739: 'AMZN:US',
 -0.5488083552676932: 'AMZN:US',
 0.305922634949743: 'AMZN:US',
 0.5556954581064577: 'AMZN:US',
 0.7161539996212608: 'AMZN:US',
 163.3869256507969: 'AMZN:US',
 164.7668700022788: 'AMZN:US',
 164.71885361188953: 'AMZN:US',
 164.69787228560168: 'AMZN:US',
 164.71907506240666: 'AMZN:US',
 2.2235951233512905: 'AMZN:US',
 1.5769454521133222: 'AMZN:US',
 1.4924948583164814: 'AMZN:US',
 1.537467828774505: 'AMZN:US',
 1.5604152426266285: 'AMZN:US',
 1.2819032059765356: 'AMZN:US',
 1.1844641682482393: 'AMZN:US',
 1.0286939229866663: 'AMZN:US',
 0.8927482951821959: 'AMZN:US',
 0.6701448720849085: 'AMZN:US',
 -0.22903885480572597: 'ASC:LN',
 -0.16432889936802358: 'ASC:LN',
 0.21282712550722824: 'ASC:LN',
 0.6494966095972378: 'ASC:LN',
 0.8401561699133191: 'ASC:LN',
 1.4090132682287244: 'ASC:LN',
 1.7793907775229556: 'ASC:LN',
 1.714324153211194: 'ASC:LN',
 1.4440850379033239: 'ASC:LN',
 1.2901561613079529: 'ASC:LN',
 1.0051036758215026: 'ASC:LN',
 0.8754518695275557

In [83]:
# calculate 5 year roic

# get max year
m_df.groupby('ticker').max()['period_end_year']


# max_years = 
# # return records from that year -5
# filt = (df.profit_end_year in )



period_end_year  gross_margin  operating_margin  net_margin  \
ticker                                                                    
AMZN:US 9            2006.5      0.174867          0.041759    0.026311   
        10           2007.5      0.156403          0.041235    0.030039   
        11           2008.5      0.139398          0.037095    0.029737   
        12           2009.5      0.123212          0.031053    0.020970   
        13           2010.5      0.107985          0.027498    0.017299   
        14           2011.5      0.095364          0.026851    0.017269   
        15           2012.5      0.082917          0.024684    0.015765   
        16           2013.5      0.087933          0.025624    0.016725   
        17           2014.5      0.092637          0.026201    0.017176   
        18           2015.5      0.097173          0.028022    0.019333   
        19           2016.5      0.104416          0.031525    0.025122   
ASC:LN  29           2007.1      0.474494          0.013308   -0.060869   
        30           2008.3      0.467720          0.070386    0.009903   
        31           2009.5      0.464736          0.077414    0.055138   
        32           2010.5      0.466126          0.073546    0.051863   
        33           2011.5      0.475269          0.070927    0.046364   
        34           2012.5      0.482552          0.067636    0.043868   
        35           2013.5      0.487724          0.062927    0.041041   
        36           2014.5      0.493187          0.055603    0.035890   
        37           2015.5      0.498837          0.051103    0.032823   
        38           2016.5      0.495204          0.047511    0.032913   
COST:US 58           2006.5      0.124899          0.027155    0.017464   
        59           2007.5      0.124990          0.027019    0.017382   
        60           2008.5      0.125069          0.027203    0.017627   
        61           2009.5      0.125151          0.027182    0.017620   
        62           2010.5      0.125702          0.027485    0.017658   
        63           2011.5      0.126714          0.027867    0.017803   
        64           2012.5      0.127655          0.028555    0.018198   
        65           2013.5      0.128279          0.029003    0.018642   
        66           2014.5      0.128534          0.029617    0.019517   
        67           2015.5      0.128853          0.030212    0.020245   
        68           2016.5      0.129168          0.030892    0.021157   

            fcf_margin    pe_ratio  fcf_yield       roic       roe       roa  \
ticker                                                                         
AMZN:US 9     0.104219  110.044317   0.051323  82.717615  0.219789  0.668422   
        10    0.111829 -177.089671   0.055405  82.805260  0.233531  0.781429   
        11    0.115584 -170.440095   0.058237  83.171908  0.235717  0.914727   
        12    0.119307 -233.409602   0.062760  83.105674  0.237081  1.312527   
        13    0.123810 -185.460137   0.063207  83.117670  0.237410  1.055871   
        14    0.132844 -178.972107   0.064281  83.139745  0.245074  0.967016   
        15    0.137843 -168.406051   0.065075   1.752749  0.241600  0.933502   
        16    0.145306 -164.492175   0.061540   1.380705  0.243035  0.954078   
        17    0.148601 -163.172597   0.060966   1.260594  0.239226  0.966848   
        18    0.161710 -162.539426   0.061627   1.215108  0.246903  1.009915   
        19    0.171476 -169.934423   0.060365   1.115280  0.248487  1.009834   
ASC:LN  29    0.052992   37.747459   0.031651   1.065156  0.218325  0.465089   
        30    0.112768   48.216771   0.042291   1.207058  0.271822  0.553265   
        31    0.130660   54.805091   0.053438   1.327421  0.312085  0.631110   
        32    0.132120   57.262059   0.055239   1.352299  0.318825  0.650069   
        33    0.128976   67.854470   0.055772   1.930888  0.319364  0.691716   
        34    0.13365

In [ ]:
# # legacy

# # perdio year/month
# df['period_end_year'] = df.period_end_date.str.split('-',expand=True)[0].astype('int64')
# df['period_end_month'] = df.period_end_date.str.split('-',expand=True)[1].astype('int64')
# df = df.drop(columns=['period_end_date'])

In [59]:
# legacy working load data function logic
# load data
data_path = Path(Path().absolute().parent, 'data', 'financials.csv')
df = pd.read_csv(data_path)

# drop empty columns
df = df[(df.period_end_date != '0')]

# rename columns
df.rename(columns={'Unnamed: 0':'ticker'}, inplace=True)

# extract month/yeat
df['period_end_year'] = df.period_end_date.str.split('-',expand=True)[0].astype('int64')
df['period_end_month'] = df.period_end_date.str.split('-',expand=True)[1].astype('int64')

# non int fileds to index 
df = df.set_index(['ticker','period_end_date', 'period_end_month', 'period_end_year', 'period_end_price'])

# convert to mil
df = df / 1000000

# return columns
df = df.reset_index()

# calculate metrics
df['fcf'] = df.cf_cfo + df.cfi_ppe_net + df.cfi_intangibles_net

df

,ticker,period_end_date,period_end_month,period_end_year,period_end_price,revenue,cogs,gross_profit,sga,rnd,...,cfi_ppe_net,cfi_acquisitions_net,cfi_investment_net,cfi_intangibles_net,cff_common_stock_net,cff_debt_net,cff_dividend_paid,cf_cff,market_cap,fcf
0,ASC:LN,2001-12,12,2001,0.160,1.702,0.705,0.997,0.000,0.0,...,-0.026,0.360,0.0,0.000,0.071,0.456,0.0,0.527,9.860800,-0.836
1,ASC:LN,2002-12,12,2002,0.053,4.104,1.941,2.163,0.000,0.0,...,-0.028,0.000,0.0,0.000,0.000,-0.049,0.0,-0.049,3.266390,-0.239
2,ASC:LN,2004-03,3,2004,0.139,0.000,0.000,0.000,0.000,0.0,...,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.000,9.449220,0.000
3,ASC:LN,2005-03,3,2005,0.530,13.518,6.928,6.590,0.000,0.0,...,-0.299,0.000,0.0,0.000,0.152,0.000,0.0,0.152,38.024320,0.903
4,ASC:LN,2006-03,3,2006,0.965,19.716,11.664,8.052,0.268,0.0,...,-0.901,0.000,0.0,0.000,0.016,0.000,0.0,0.016,69.388325,1.682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,ZLNDY:US,2017-12,12,2017,26.810,5312.426,2993.609,2318.817,2099.053,0.0,...,-223.787,-41.302,0.0,-64.852,-8.758,-3.787,0.0,-12.544,13255.185720,-59.408
78,ZLNDY:US,2018-12,12,2018,13.060,6129.579,3534.699,2594.881,2466.553,0.0,...,-190.216,-8.077,0.0,-59.499,-83.163,-3.185,0.0,-86.348,6474.377460,-7.622
79,ZLNDY:US,2019-12,12,2019,25.120,7202.778,4138.111,3064.667,912.333,0.0,...,-234.667,-1.889,0.0,-80.889,-0.333,-3.111,0.0,-63.667,12493.733440,48.000
80,ZLNDY:US,2020-12,12,2020,55.230,9710.462,5581.265,4129.197,1192.336,0.0,...,-176.277,-31.022,0.0,-88.200,67.032,1662.044,0.0,1647.324,27990.287850,377.129


In [ ]:


# # create index logic

# # generate list of numbers matched with tickers
# tmap = {v:k for k,v in enumerate(df.ticker.unique())}

# # map the list and insert to df
# t_index = df.ticker.map(tmap)
# df.insert(0,'index',t_index)



In [ ]:
# # create id mapping

# tmap = {v:k for k,v in enumerate(df.ticker.unique())}

# tindex = df.ticker.map(tmap)
# df.insert(0,t_index)

In [60]:
# # legacy

# data_path = Path(Path().absolute().parent, 'data', 'financials.csv')
# df = pd.read_csv(data_path)
# df = df.dropna()
# df['period_end_year'] = df.period_end_date.str.split('-',expand=True)[0].astype('int64')
# df['period_end_month'] = df.period_end_date.str.split('-',expand=True)[1].astype('int64')
# df = df.drop(columns=['period_end_date'])

# # convert to mil
# df = df / 1000000
# df = df.dropna(axis=0,how='all')
# df = df.reset_index()

# # add calculated features
# # fcf need to be minuses cfo has negative values
# df['fcf'] = df.cf_cfo + df.cfi_ppe_net + df.cfi_intangibles_net

In [ ]:
# # legacy load data functioun
# # load data
#     data_path = Path(Path().absolute().parent, 'data', 'financials.csv')
#     df = pd.read_csv(data_path)

#     # rename columns
#     df.rename(columns={'Unnamed: 0':'ticker'}, inplace=True)
#     # put non int data in the index 
#     df = df.set_index(['ticker','period_end_date', 'period_end_price'])

#     # convert to mil
#     df = df / 1000000
#     df = df.dropna(axis=0,how='all')

#     # return columns
#     df = df.reset_index()

#     # calculate metrics
#     df['fcf'] = df.cf_cfo + df.cfi_ppe_net + df.cfi_intangibles_net